## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.74,-97.86,-4.00,83,20,5.82,few clouds
1,1,Miri,MY,4.41,114.01,78.80,94,40,6.93,scattered clouds
2,2,Bengkulu,ID,-3.80,102.27,75.79,86,83,5.23,light rain
3,3,Iranshahr,IR,27.20,60.68,75.20,14,0,4.70,clear sky
4,4,Busselton,AU,-33.65,115.33,54.00,95,62,17.65,broken clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Busselton,AU,-33.65,115.33,54.00,95,62,17.65,broken clouds
5,5,Mount Gambier,AU,-37.83,140.77,46.40,100,0,2.51,clear sky
9,9,Faya,SA,18.39,42.45,64.40,63,40,6.93,scattered clouds
10,10,Kruisfontein,ZA,-34.00,24.73,70.00,58,0,5.99,clear sky
11,11,Itoman,JP,26.12,127.67,68.00,60,75,17.22,broken clouds
19,19,Christchurch,NZ,-43.53,172.63,55.00,81,91,11.41,overcast clouds
22,22,Bluff,NZ,-46.60,168.33,52.00,85,100,6.76,overcast clouds
24,24,New Norfolk,AU,-42.78,147.06,48.99,64,3,2.68,clear sky
25,25,Port Alfred,ZA,-33.59,26.89,68.29,67,0,12.59,clear sky
27,27,Ushuaia,AR,-54.80,-68.30,44.60,75,75,21.92,broken clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                201
City                   201
Country                200
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Current Description    201
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Busselton,AU,54.00,broken clouds,-33.65,115.33,
5,Mount Gambier,AU,46.40,clear sky,-37.83,140.77,
9,Faya,SA,64.40,scattered clouds,18.39,42.45,
10,Kruisfontein,ZA,70.00,clear sky,-34.00,24.73,
11,Itoman,JP,68.00,broken clouds,26.12,127.67,
19,Christchurch,NZ,55.00,overcast clouds,-43.53,172.63,
22,Bluff,NZ,52.00,overcast clouds,-46.60,168.33,
24,New Norfolk,AU,48.99,clear sky,-42.78,147.06,
25,Port Alfred,ZA,68.29,clear sky,-33.59,26.89,
27,Ushuaia,AR,44.60,broken clouds,-54.80,-68.30,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found for "+row["City"])
        pass
        

Hotel not found for Marzuq
Hotel not found for Abu Samrah
Hotel not found for Canakkale


In [58]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotel_df = hotel_df.copy()
clean_hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df.dropna(subset=["Hotel Name"], inplace=True)

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Busselton,AU,54.0,broken clouds,-33.65,115.33,Observatory Guest House
5,Mount Gambier,AU,46.4,clear sky,-37.83,140.77,Commodore on the Park
9,Faya,SA,64.4,scattered clouds,18.39,42.45,مقبرة ال مجمل
10,Kruisfontein,ZA,70.0,clear sky,-34.00,24.73,Oyster Bay House Rental
11,Itoman,JP,68.0,broken clouds,26.12,127.67,Southern Beach Hotel & Resort Okinawa
...,...,...,...,...,...,...,...
657,Tezu,IN,64.4,clear sky,27.92,96.17,Circuit House
663,Russell,US,59.0,light rain,32.35,-85.20,Macs
666,Winchester,GB,48.2,broken clouds,51.07,-1.32,Lainston House
671,San Buenaventura,MX,63.0,clear sky,27.08,-101.53,Hotel Gran Plaza


In [59]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [68]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

#hotel_info

In [69]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))